In [ ]:
def get_marketaux_news(company_name: str) -> str:

    import requests
    
    API_KEY = "your_marketaux_api_key"
    url = "https://api.marketaux.com/v1/news/all"
    params = {
        "api_token": "A3k62V7qsceE4p8tM4hTO053p1RxjcVJ8xqdy0zV",
        "symbols": company_name,
        "language": "en",
        "limit": 5
    }
    response = requests.get(url, params=params)
    data = response.json()
    if "data" in data:
        return [article["url"] for article in data["data"]]
    else:
        return []


company_name = "TSLA"

news_urls = get_marketaux_news(company_name)
print(news_urls)

['https://finance.yahoo.com/news/prediction-artificial-intelligence-ai-company-220000588.html', 'https://finance.yahoo.com/news/tesla-inc-tsla-among-best-202341203.html', 'https://seekingalpha.com/news/4426704-tesla-shorts-make-10b-ytd-short-interest-rebounds-to-81m-shares']


In [11]:
def scrape_article(url: str) -> str:
    from newspaper import Article

    try:
        article = Article(url)
        article.download()
        article.parse()
        return {
            "title": article.title,
            "content": article.text,
        }
    except Exception as e:
        print(f"Error scraping article: {e}")
        return {
            "title": "",
            "content": "",
        }

articles = [scrape_article(url) for url in news_urls]

Error scraping article: Article `download()` failed with 403 Client Error: Forbidden for url: https://seekingalpha.com/news/4426704-tesla-shorts-make-10b-ytd-short-interest-rebounds-to-81m-shares on URL https://seekingalpha.com/news/4426704-tesla-shorts-make-10b-ytd-short-interest-rebounds-to-81m-shares


In [12]:
for article in articles:
    print(f"title:{article['title']} len:{len(article['content'])}")

title:Prediction: This Artificial Intelligence (AI) Company Will Be the Biggest Beneficiary of Self-Driving Vehicles (Hint: It's Not Tesla) len:2586
title:Tesla, Inc. (TSLA): Among the Best Robotics Stocks to Buy According to Billionaires len:2983
title: len:0


In [50]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import BaseLLM
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

def get_summary_from_llm(llm: BaseLLM, config: dict, article_title: str, article_content: str) -> str:
    prompt = f"""you are a professional journalist who has been asked to analyze the following article and provide a summary, sentiment analysis, and key points.
        Analyze the following article and provide a summary, sentiment analysis, and key points:

        Title: {article_title}
        Content: {article_content}

        Please provide your analysis in the following format:
        
        Sentiment: The overall sentiment score (positive:10, negative:1, or neutral:5)
        Summary: A concise summary of the article (2-3 sentences)
        
    """

    return llm.invoke(input=prompt, config=config).content


In [58]:

def get_human_message(article_title: str, article_content: str) -> str:
    prompt = f"""you are a professional journalist who has been asked to analyze the following article and provide a summary, sentiment analysis, and key points.
        Analyze the following article and provide a summary, sentiment analysis, and key points:

        Title: {article_title}
        Content: {article_content}

        Please provide your analysis in the following format:
        
        Sentiment: The overall sentiment score (positive:10, negative:1, or neutral:5)
        Summary: A concise summary of the article (2-3 sentences)
        Key Points: The key points of the article (3-5 bullet points)
        
    """
    return prompt

In [59]:
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import MemorySaver
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

config = {"configurable": {"thread_id": "1234"}}
llm = init_chat_model("gpt-4o-mini", model_provider="openai", store=True)

workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
    response = llm.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

i = 0
for article in articles:
    i = i+1
    print(f"\n--{i}--\n")

    if article['content']:
        print(f"Title: {article['title']}")
        
        input_messages = [HumanMessage(get_human_message(article['title'], article['content']))]
        output = app.invoke({"messages": input_messages}, config)
        output["messages"][-1].pretty_print()
        
    else:
        print(f"Skipping article '{article['title']}' due to empty content.")


/var/folders/v_/495_mfbd4rv76fxlb3w_3hs80000gn/T/ipykernel_92536/2875842115.py:7: UserWarning: WARNING! store is not default parameter.
                store was transferred to model_kwargs.
                Please confirm that store is what you intended.
  llm = init_chat_model("gpt-4o-mini", model_provider="openai", store=True)



--1--

Title: Prediction: This Artificial Intelligence (AI) Company Will Be the Biggest Beneficiary of Self-Driving Vehicles (Hint: It's Not Tesla)
================================== Ai Message ==================================

**Sentiment:** Positive (8/10)

**Summary:** The article highlights Nvidia's significant role in the automotive industry, positioning it as a leading beneficiary of advancements in self-driving vehicles, surpassing Tesla. It discusses Nvidia's innovative technology offerings, including AI-driven platforms for vehicle simulation and manufacturing efficiency, and outlines the company’s various partnerships with major automotive players.

**Key Points:**
- Nvidia is emerging as a powerful player in the automotive sector, providing AI solutions to improve safety and manufacturing processes.
- The company’s Omniverse platform allows for the creation of digital twins to simulate vehicle functionality in different environments.
- Nvidia has partnered with General Mo

In [66]:
prompt = "What's your overall sentiment about these articles? (positive:10, negative:1, or neutral:5). Only Score/10 is required."

input_messages = [HumanMessage(prompt)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()


================================== Ai Message ==================================

8/10
